In [63]:
# 모듈 임포트

from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
import pandas as pd


# 연결 주소 설정

url = 'https://www.starbucks.co.kr/store/store_map.do'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)


# 웹페이지에서 단추를 눌러서 원하는 자료로 이동

area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR,area_btn).click()

time.sleep(2)
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR,seoul_btn).click()

time.sleep(2)
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR,all_btn).click()


# 웹 페이지 긁어오기

txt = driver.page_source
html = bs(txt)

# 전체 코드에서 필요한 부분만 추출

shops = html.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')


# 한개 지점만 일단 추출

shop = shops[0]
shop


# 필요한 데이터만 추출

# 위도 : data-lat 딕셔너리 형태로 추출 즉 키값 밸류값
lat = shop['data-lat']

# 경도 : data-long 딕셔너리 형태로 추출 즉 키값 밸류값
lng = shop['data-long']

# 지점명 : data-name 딕셔너리 형태로 추출 즉 키값 밸류값
name = shop['data-name']

# <p class="result_details">서울특별시 마포구 양화로 105 (서교동)<br/>1522-3232</p>

# 주소 : 딕셔너리 형태가 아니기 때문에 select으로 추출
addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]

# 전화번호 : 주소와 전화번호는 묶여 있으므로 복잡하게 추출
tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

# 매장종류 : 딕셔너리 형태가 아니기 때문에 select으로 추출
stype = shop.select('i.pin_general')[0].text


# 모든 지점의 필요 데이터 추출

starbucks_list = []

for shop in shops:
    lat = shop['data-lat']
    lng = shop['data-long']
    name = shop['data-name']
    addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    stype = shop.select('i')[0].text

    time.sleep(0.01)
    starbucks_list.append([name, lat, lng, addr, tel, stype])

print(starbucks_list)


# 데이터 프레임으로 전환
df = pd.DataFrame(starbucks_list, columns = ['매장명','위도','경도','주소','전화번호','매장종류'])
df

C:\Users\tlaru\AppData\Local\Temp\ipykernel_16952\2049587862.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


IndexError: list index out of range

In [64]:
############################################################
#                          데이터 분석                      #
############################################################

#!pip install folium


import folium


# 기본 지도 그리기 연습

starmap = folium.Map(
    location=[37.5666805,126.9784147],  # 구글에서 검색한 서울 시청 좌표
    zoom_start=11,
    tiles='Stamen Terrain'
)

folium.CircleMarker(
    location=[37.5666805,126.9784147],
    fill = True
).add_to(starmap)


starmap




# 스타벅스 600개 지점 지도 그리기

import folium

starmap = folium.Map(
    location=[37.5666805,126.9784147],  # 구글에서 검색한 서울 시청 좌표
    zoom_start=11,
    tiles='Stamen Terrain'
)

# 데이터 프레임에서 위도와 경도 값을 가져와서 600번 반복

for idx in df.index:
    lat = df.loc[idx,'위도']
    lng = df.loc[idx,'경도']

    folium.Marker(
        location=[lat,lng],
        fill = True
    ).add_to(starmap)


starmap